In [ ]:
import tensorflow as tf 
import os as os 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 

In [ ]:
data_dir = '/content/drive/MyDrive/AutismDataset/data' 
image_exts = ['jpeg','jpg', 'bmp', 'png'] 
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)): 
        image_path = os.path.join(data_dir, image_class, image) 
        try: 
            img = cv2.imread(image_path) 
            tip = imghdr.what(image_path) 
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path)) 
                os.remove(image_path) 
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt 

data = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/AutismDatas
 et/data') 
data_iterator = data.as_numpy_iterator() 
batch = data_iterator.next() 
batch[0].shape 
fig, ax = plt.subplots(ncols=4, figsize=(20,20)) 
for idx, img in enumerate(batch[0][:4]): 
    ax[idx].imshow(img.astype(int)) 
    ax[idx].title.set_text(batch[1][idx]) 

In [ ]:
data = data.map(lambda x,y: (x/255, y)) 
data.as_numpy_iterator().next() 
batch[0].max() 
len(data)

In [ ]:
rain_size = int(len(data)*.75)+1 
val_size = int(len(data)*.15) 
test_size = int(len(data)*.1) 
 
train = data.take(train_size) 
val = data.skip(train_size).take(val_size) 
test = data.skip(train_size+val_size).take(test_size) 

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, 
Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
 
# Create a Sequential model 
model = Sequential() 
 
model.add(Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))

In [ ]:
model.add(MaxPooling2D()) 
model.add(BatchNormalization()) 
 
model.add(Conv2D(32, (3, 3), 1, activation='relu')) 
model.add(MaxPooling2D()) 
model.add(BatchNormalization()) 
 
model.add(Conv2D(16, (3, 3), 1, activation='relu')) 
model.add(MaxPooling2D()) 
model.add(BatchNormalization()) 
 
model.add(Flatten()) 
 
model.add(Dense(256, activation='relu')) 
model.add(Dropout(0.5))  # Adding Dropout for regularization 
model.add(BatchNormalization()) 
 
model.add(Dense(1, activation='sigmoid')) 
 
# Compile the model 
model.compile(optimizer=Adam(lr=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy']) 
 
# Load your training and validation data using train_datagen 
 
# Set up callbacks 
early_stop = EarlyStopping(monitor='val_loss', patience=5, 
restore_best_weights=True) 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, 
min_lr=1e-6) 
 
# Train the model 
 
model.summary() 
logdir='logs' 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) 
 
history = model.fit( 
    train, 
    epochs=50, 
    validation_data=val, 
    callbacks=[early_stop, reduce_lr]) 

In [ ]:
fig = plt.figure() 
plt.plot(history.history['loss'], color='red', label='loss') 
plt.plot(history.history['val_loss'], color='orange', label='val_loss') 
fig.suptitle('Loss', fontsize=20) 
plt.legend(loc="upper left") 
plt.show() 
 
fig = plt.figure() 
plt.plot(history.history['accuracy'], color='teal', label='accuracy') 
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy') 
fig.suptitle('Accuracy', fontsize=20) 
plt.legend(loc="upper left") 
plt.show() 

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy 
 
pre = Precision() 
re = Recall() 
acc = BinaryAccuracy() 
 
for batch in test.as_numpy_iterator(): 
    X, y = batch 
    z = model.predict(X) 
    pre.update_state(y, z) 
    re.update_state(y, z) 
    acc.update_state(y, z) 
 
print(pre.result(), re.result(), acc.result()) 

In [ ]:
from tensorflow.keras.models import load_model 
 
model.save(os.path.join('DL models','autism(1).h5')) 
 
new_model = load_model('autism(1).h5') 